In [46]:
import pandas as pd
import numpy as np
import nltk
import string
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, GPT2Tokenizer, AdamW, BertTokenizer,TFGPT2ForSequenceClassification
from transformers import TFBertForSequenceClassification,TFBertTokenizer,pipeline
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
import torch
from torch.optim import AdamW
from transformers import DataCollatorWithPadding

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [8]:
!mkdir -p ~/.kaggle|
!cp kaggle.json ~/.kaggle/

/bin/bash: -c: line 2: syntax error: unexpected end of file


In [9]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 47% 12.0M/25.7M [00:00<00:00, 126MB/s]
100% 25.7M/25.7M [00:00<00:00, 144MB/s]


In [10]:
!unzip -q imdb-dataset-of-50k-movie-reviews.zip

In [11]:
main_data = pd.read_csv('/content/IMDB Dataset.csv')

In [12]:
main_data['sentiment'].replace({'positive':1,'negative':0},inplace=True)

In [13]:
data  = main_data.iloc[:2500]

In [14]:
data.shape

(2500, 2)

In [15]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [16]:
data['sentiment'].replace({'positive':1,'negative':0},inplace=True)

<ipython-input-16-f0e6f0734aef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'].replace({'positive':1,'negative':0},inplace=True)


In [17]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [18]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [19]:
data.duplicated().sum()

0

In [20]:
sw = set(stopwords.words('english'))
stemmer = WordNetLemmatizer()

In [21]:
def fun2(x):
    # Remove HTML tags
    x = re.sub(r'<.*?>', '', x)

    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9]', ' ', x)

    # Replace single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Replace single characters from the start
    text = re.sub(r'^[a-zA-Z]\s+', ' ', text)

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # Tokenize, lowercase, remove stop words, and lemmatize
    tokens = word_tokenize(text)
    tokens = [i.lower() for i in tokens if i not in sw]
    tokens = [i for i in tokens if i not in string.punctuation]
    tokens = [stemmer.lemmatize(token) for token in tokens]
    x = ' '.join(tokens)

    return x

In [22]:
data['review'] = data['review'].apply(fun2)

<ipython-input-22-2c1159babae6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(fun2)


In [23]:
data

,review,sentiment
0,one reviewer mentioned watching 1 oz episode h...,1
1,wonderful little production the filming techni...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake think zombie ...,0
4,petter mattei love time money visually stunnin...,1
...,...,...
2495,another great movie costa gavras it great pres...,0
2496,though structured totally different book tim k...,1
2497,handsome dashing british airline pilot george ...,1
2498,this film breech fine line satire silliness wh...,0


## Model Buinding

In [24]:
from transformers import AutoTokenizer
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

In [25]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [114]:
x = data['review']
y = data['sentiment']

In [115]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [116]:
x_train_tok = tokenizer(x_train.values.tolist(), truncation=True,padding='max_length',return_tensors="np")
x_test_tok = tokenizer(x_test.values.tolist(), truncation=True,padding='max_length',return_tensors="np")
#tokenized_datasets

In [117]:
x_train_tok['input_ids'][:10]

array([[  101,  1122,  1121, ...,     0,     0,     0],
       [  101,  1142, 22302, ...,     0,     0,     0],
       [  101,   122,  1645, ...,     0,     0,     0],
       ...,
       [  101,  1176,  2166, ...,     0,     0,     0],
       [  101,  2121,  5351, ...,     0,     0,     0],
       [  101, 12187,  5694, ...,     0,     0,     0]])

In [118]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices(
    ({'input_ids': x_train_tok['input_ids'], 'token_type_ids': x_train_tok['token_type_ids'], 'attention_mask': x_train_tok['attention_mask']}, np.array(y_train))
).shuffle(10).batch(8)

eval_tf_dataset = tf.data.Dataset.from_tensor_slices(
    ({'input_ids': x_test_tok['input_ids'], 'token_type_ids': x_test_tok['token_type_ids'], 'attention_mask': x_test_tok['attention_mask']}, np.array(y_test))
).batch(8)

In [119]:
train_tf_dataset

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [120]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [121]:
# last_layer = model.layers[-1]

# # Change the activation function to sigmoid and set the number of neurons to 1
# last_layer.activation = tf.keras.activations.softmax
# last_layer.units = 2

In [122]:
# Assuming model is your TFAutoModelForSequenceClassification model
last_layer = model.layers[-1]

# Check the activation function
activation_function = last_layer.activation.__name__
print(f"The activation function of the last layer is: {activation_function}")

# Check the number of neurons
num_neurons = last_layer.units
print(f"The number of neurons in the last layer is: {num_neurons}")

The activation function of the last layer is: linear
The number of neurons in the last layer is: 2


In [123]:
model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 108311810 (413.18 MB)
Trainable params: 108311810 (413.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [124]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [125]:
model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=10)

Epoch 1/10
  8/250 [..............................] - ETA: 3:33 - loss: 0.7000 - accuracy: 0.5000

KeyboardInterrupt: ignored

In [45]:
model.save('bert_fine_tune', save_format='tf')

KeyboardInterrupt: ignored

In [46]:
model.evaluate(eval_tf_dataset)

63/63 [==============================] - 21s 332ms/step - loss: 0.3214 - accuracy: 0.8700


[0.32140690088272095, 0.8700000047683716]

In [86]:
test = main_data.iloc[2987]

In [87]:
test_x = test['review']
test_y = test['sentiment']

In [88]:
test_x = fun2(test_x)

In [89]:
test_tokenized = tokenizer(test_x, padding='max_length', truncation=True)

In [90]:
len(test_tokenized['input_ids'])

512

In [91]:
input_ids = tf.constant([test_tokenized['input_ids']])
token_type_ids = tf.constant([test_tokenized['token_type_ids']])
attention_mask = tf.constant([test_tokenized['attention_mask']])

In [92]:
input_data = {
    'input_ids': input_ids,
    'token_type_ids': token_type_ids,
    'attention_mask': attention_mask
}

In [93]:
#input_data

In [94]:
pred = model.predict(input_data)['logits']

1/1 [==============================] - 0s 114ms/step


In [95]:
pred

array([[-1.4277854,  1.4665383]], dtype=float32)

In [96]:
pred = tf.nn.softmax(pred)

In [102]:
pred

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.05243487, 0.9475651 ]], dtype=float32)>

In [103]:
value = np.argmax(pred)

In [104]:
value,test_y

(1, 1)

### Using Pipeline

In [108]:
sentiment_analysis = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Example usage:
text = "it is good and not"

clean_text = fun2(text)

# Make predictions
result = sentiment_analysis(clean_text)

# Print the result
print(result)

[{'label': 'LABEL_1', 'score': 0.9557428956031799}]


In [ ]:
#sentiment_analysis.save_pretrained('pipeline')